In [50]:
import numpy as np
import pickle
import lowda
import dill
import arepo

In [40]:
import importlib
importlib.reload(lowda)

<module 'lowda' from '/n/holylfs05/LABS/hernquist_lab/Users/abeane/GSEgas/plots/lowda.py'>

In [41]:
basepath='/n/holylfs05/LABS/hernquist_lab/Users/abeane/GSEgas'
name = 'MW4iso_fg0.2_MHG0.25_RC9_vphi0.1'
lvl = 'lvl4-noB'
output_path = basepath + '/runs/' + name + '/' + lvl + '/output'

In [55]:
sn = arepo.Snapshot(output_path, 10, combineFiles=True)
sn.addField('Temperature', [1,0,0,0,0,0])
sn.Temperature[:] = np.random.rand(sn.NumPart_Total[0])

In [57]:
sn.ntypes

6

In [42]:
parttype = [0, 2, 4]
fields = None
idx = 0

out = lowda._load_snap(output_path, idx, parttype, fields)

In [43]:
# dill.dump(out, open('/scratch/test.p', 'wb'))

In [44]:
# outt = dill.load(open('/scratch/test.p', 'rb'))

In [45]:
pickle.dump(out, open('/scratch/test.p', 'wb'))

In [46]:
outt = pickle.load(open('/scratch/test.p', 'rb'))

In [24]:
pickle.dump(out['sn'].header, open('/scratch/test.p', 'wb'))

In [25]:
outt = pickle.load(open('/scratch/test.p', 'rb'))

RecursionError: maximum recursion depth exceeded

In [28]:
sn.__getstate__?

Object `sn.__getstate__` not found.


In [34]:
sn = out['sn']
snf = snap(sn)

In [35]:
pickle.dump(snf, open('/scratch/test.p', 'wb'))

In [36]:
snt = pickle.load(open('/scratch/test.p', 'rb'))

In [37]:
snt.part0.Coordinates

array([[300.        , 300.        , 300.        ],
       [297.37298584, 300.2114563 , 300.0199585 ],
       [301.78240967, 296.81924438, 300.14727783],
       ...,
       [337.6114502 , 322.73294067, 308.03738403],
       [475.02960205, 330.38162231, 199.24707031],
       [432.42095947, 209.29766846, 490.61062622]])

In [60]:
from multiprocessing.managers import BaseManager
from multiprocessing import Process
import multiprocessing

In [62]:
def load_and_process_file(filename, shared_dict_name):
    # Load the file
    data = np.random.rand(10000)

    # Now, store the processed data in the shared dictionary
    with multiprocessing.managers.BaseManager() as manager:
        manager.register('get_shared_dict')
        manager.connect()
        shared_dict = manager.get_shared_dict()
        shared_dict[filename] = data

class SharedDictManager(BaseManager): pass

def Manager():
    m = SharedDictManager()
    m.start()
    return m

SharedDictManager.register('get_shared_dict', dict, multiprocessing.managers.DictProxy)

manager = Manager()
shared_dict = manager.get_shared_dict()


In [65]:
filenames = ['0', '1']#, '2', '3', '4', '5', '6', '7']
processes = []
for fname in filenames:
    p = Process(target=load_and_process_file, args=(filenames, shared_dict._id))
    p.start()
    processes.append(p)

for p in processes:
    p.join()
    
print(shared_dict)

Process Process-11:
Process Process-12:
Traceback (most recent call last):
  File "/n/home01/abeane/.conda/envs/note/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/n/home01/abeane/.conda/envs/note/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/n/home01/abeane/.conda/envs/note/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/n/home01/abeane/.conda/envs/note/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_193598/3596152153.py", line 9, in load_and_process_file
    shared_dict = manager.get_shared_dict()
  File "/n/home01/abeane/.conda/envs/note/lib/python3.10/multiprocessing/managers.py", line 723, in temp
    token, exp = self._create(typeid, *args, **kwds)
  File "/tmp/ipykernel_193598/3596152153.py", line 9, in load

{}
